# Neural Network used for the paper Suelves. et al (2022), in preparation. The repository includes the training saved weights, but the training+test dataset should be retrieved from Casjobs using the indications of the paper

## For any questions, please do not hesitate to contact me on luis.suelves@ncbj.gov.pl or luiseduardosuelves@gmail.com

In [ ]:
from platform import python_version 

print(python_version()) # The versions used by L.E. Suelves for the paper was 3.8.5

import tensorflow as tf
# import tensorflow
tf.keras.backend.set_floatx('float32')

from astropy.io import fits
from astropy.table import Table

import numpy as np

import matplotlib.pyplot as plt
import random

from sklearn.model_selection import StratifiedKFold

In [ ]:
#Check if GPUs. If there are, some code to fix cuDNN bugs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
else:
    print('No GPU')

In [ ]:
# name_file = 

USE_COLUMNS = ['skyErr_u', 'skyErr_g','skyErr_r', 'skyErr_i', 'skyErr_z']

USE_COLUMNS_short = ['skyE-u', 'skyE-g', 'skyE-r', 'skyE-i', 'skyE-z']


CLASS_NAMES = ['merger', 'nonmerger'] # only used for the numbering!!
NO_CLASS = len(CLASS_NAMES)
print(NO_CLASS)
CLASS_COLUMN = 'cl'

EPOCHS = 1500
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 10000

STEPS_PER_EPOCH = 1
STEPS_PER_VALID_EPOCH = 1
STEPS_PER_TEST_EPOCH = 1

train_photo_count = 0
valid_photo_count = 0
test_photo_count = 0

In [ ]:
def prepare_data(file_path, class_column, class_number, use_columns, err_cols=False, multi_cols=None):
    ### Convert the data from the dataset .fits file into a table
    table = Table.read(file_path)

    labels = tf.one_hot(table[class_column].data, class_number)
    data = []
    for column in use_columns:
        data.append(table[column].data)
    data = np.array(data)    
    data = data.T

    return np.hstack((data,labels))

pre_error_tab = prepare_data(name_file, CLASS_COLUMN, NO_CLASS, USE_COLUMNS)

In [ ]:
def kfold_iteration(table, set_type):
    ### Creates the tensorflow data using the table obtained in prepare_data()
    ### It shuffles the batchs randomly, which is useful for training
    data = table[:,:5]
    labels = table[:,5:]
    
    if 'Train' in set_type:
        global train_photo_count
        global STEPS_PER_EPOCH
        train_photo_count = len(data)
        STEPS_PER_EPOCH = np.ceil(train_photo_count/BATCH_SIZE).astype(int)
        batch_size = BATCH_SIZE
    elif 'Valid' in set_type:
        global valid_photo_count
        global STEPS_PER_VALID_EPOCH
        valid_photo_count = len(data)
        #STEPS_PER_VALID_EPOCH = np.floor(valid_photo_count/BATCH_SIZE).astype(int)
        STEPS_PER_VALID_EPOCH = 1
        batch_size = valid_photo_count
    
    data = np.log10(data)
#     print(data[0])
        
    ds = tf.data.Dataset.from_tensor_slices((data, labels))
#     print(next(iter(ds.as_numpy_iterator())))
    ds = ds.shuffle(SHUFFLE_BUFFER_SIZE).batch(batch_size)
    return ds

In [ ]:
kfold_iteration(pre_error_tab,set_type='Valid')

In [ ]:
def kfold_iteration_noshufle(table, set_type):
    ### Creates the tensorflow data using the table obtained in prepare_data()
    ### It does not shuffle the batchs randomly, avoiding problems when plotting the dataset
    data = table[:,:5]
    labels = table[:,5:]
    
    if 'Train' in set_type:
        global train_photo_count
        global STEPS_PER_EPOCH
        train_photo_count = len(data)
        STEPS_PER_EPOCH = np.ceil(train_photo_count/BATCH_SIZE).astype(int)
        batch_size = BATCH_SIZE
    elif 'Valid' in set_type:
        global valid_photo_count
        global STEPS_PER_VALID_EPOCH
        valid_photo_count = len(data)
        #STEPS_PER_VALID_EPOCH = np.floor(valid_photo_count/BATCH_SIZE).astype(int)
        STEPS_PER_VALID_EPOCH = 1
        batch_size = valid_photo_count
    
    data = np.log10(data)
    print(data[0])
    ds = tf.data.Dataset.from_tensor_slices((data, labels))
    print(next(iter(ds.as_numpy_iterator())))
    ds = ds.batch(batch_size)
    return ds

In [ ]:
kfold_iteration_noshufle(pre_error_tab,set_type='Train')

In [ ]:
class photo_model(tf.keras.Model):
    def __init__(self):
        ## Im trying to name each layer and see if the loading is managed through it
        super(photo_model, self).__init__()
        self.drop_rate = 0.1
        
        self.fuco1 = tf.keras.layers.Dense(16,name='dens_1')
        self.batn1 = tf.keras.layers.BatchNormalization(name='btchn_1')
        self.drop1 = tf.keras.layers.Dropout(self.drop_rate,name='drop_1')
        
        self.fuco5 = tf.keras.layers.Dense(16,name='dens_2')
        self.batn5 = tf.keras.layers.BatchNormalization(name='btchn_2')
        self.drop5 = tf.keras.layers.Dropout(self.drop_rate,name='drop_1')
        
        self.y_out = tf.keras.layers.Dense(NO_CLASS, activation='softmax',name='out')
        
    def call(self, x, training=True):
        
        x = self.fuco1(x)
        x = self.batn1(x)
        x = tf.keras.activations.relu(x)
        x = self.drop1(x, training=training)
        
        x = self.fuco5(x)
        x = self.batn5(x)
        x = tf.keras.activations.relu(x)
        x = self.drop5(x, training=training)
        
        return self.y_out(x)

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

val_loss = tf.keras.metrics.Mean(name='val_loss')
val_accuracy = tf.keras.metrics.CategoricalAccuracy(name='val_accuracy')

@tf.function
def train_step(data, labels):
    '''labels shoule be one_hot'''
    with tf.GradientTape() as tape:
        pred = model(data)
        loss = total_loss(labels, pred)
        mean_loss = tf.reduce_mean(loss)

    #Update gradients and optimize
    grads = tape.gradient(mean_loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    
    #tf statistics tracking
    train_loss(mean_loss)
    train_accuracy(labels, pred)

@tf.function
def val_step(data, labels):
    '''labels should be one_hot'''
    pred = model(data, training=False)
    v_loss = total_loss(labels, pred)
    mean_v_loss = tf.reduce_mean(v_loss)

    #tf statistics tracking
    val_loss(mean_v_loss)
    val_accuracy(labels, pred)
    return pred

In [ ]:
# model = photo_model()

total_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

In [ ]:
print('number of fold-training epochs',EPOCHS)
peak = [0, 0, 100]
savedpeak = []

kt_los = []
kt_acc = []
kv_los = []
kv_acc = []

template = 'Epoch {}\nTrain Loss: {:.3g}, Train Accuracy: {:.3g}\nValid Loss: {:.3g}, Valid Accuracy: {:.3g}'

k = 5
skfold = StratifiedKFold(k, False)
sk_data = pre_error_tab[:,:5]
sk_labels = pre_error_tab[:,5]

www = []
fold_no = 1
for train_sk, valid_sk in skfold.split(sk_data,sk_labels):
    print('fold number: {}, train: {}, valid:{}' .format(fold_no,len(train_sk), len(valid_sk)))
    train_ds = kfold_iteration(pre_error_tab[train_sk],set_type='Train')
    valid_ds = kfold_iteration(pre_error_tab[valid_sk],set_type='Valid')
    print(valid_ds)
    t_los = []
    t_acc = []
    v_los = []
    v_acc = []
    www_temp = []
    tf.keras.backend.clear_session()
    peak = [0, 0, 100]
    
    ## Here I create the full model again
    ## It also semed necessary to initialize again the train and validation \
    ##    steps that will be used in the following Epoch loop!
    
    model = photo_model()
    train_loss = tf.keras.metrics.Mean(name='train_loss')
    train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

    val_loss = tf.keras.metrics.Mean(name='val_loss')
    val_accuracy = tf.keras.metrics.CategoricalAccuracy(name='val_accuracy')

    @tf.function
    def train_step(data, labels):
        '''labels shoule be one_hot'''
        with tf.GradientTape() as tape:
            pred = model(data)
            loss = total_loss(labels, pred)
            mean_loss = tf.reduce_mean(loss)

        #Update gradients and optimize
        grads = tape.gradient(mean_loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
    
        #tf statistics tracking
        train_loss(mean_loss)
        train_accuracy(labels, pred)

    @tf.function
    def val_step(data, labels):
        '''labels should be one_hot'''
        pred = model(data, training=False)
        v_loss = total_loss(labels, pred)
        mean_v_loss = tf.reduce_mean(v_loss)

        #tf statistics tracking
        val_loss(mean_v_loss)
        val_accuracy(labels, pred)
        return pred
    
    ## Epoch loop
    for epoch in range(0, EPOCHS):
    
        train_loss.reset_states()
        train_accuracy.reset_states()
        val_loss.reset_states()
        val_accuracy.reset_states()
        
        #Train
        for step in range(0, STEPS_PER_EPOCH):
            x_batch, y_batch = next(iter(train_ds))
            train_step(x_batch, y_batch)
    
        #Validate  
        y_val_all = None
        val_pred = None
        for step in range(0, STEPS_PER_VALID_EPOCH):
            x_val, y_val = next(iter(valid_ds))
            if y_val_all is None:
                y_val_all = y_val
            else:
                y_val_all = np.vstack((y_val_all, y_val))
            pred = val_step(x_val, y_val)
            if val_pred is None:
                val_pred = pred
            else:
                val_pred = np.vstack((val_pred, pred))
    
        if epoch%100 == 0:
            print(template.format(epoch+1,
                                  train_loss.result(), train_accuracy.result(),
                                  val_loss.result(), val_accuracy.result()))
    
        t_los.append(train_loss.result())
        t_acc.append(train_accuracy.result())
        v_los.append(val_loss.result())
        v_acc.append(val_accuracy.result())
        
        kt_los.append(train_loss.result())
        kt_acc.append(train_accuracy.result())
        kv_los.append(val_loss.result())
        kv_acc.append(val_accuracy.result())
    
        if val_loss.result() <= peak[2] and val_accuracy.result() >= peak[1]:
            peak[0] = epoch+1
            peak[1] = val_accuracy.result()
            peak[2] = val_loss.result()
            savedpeak.append((peak[0],np.round(peak[1],3),np.round(peak[2],3),))
            www_temp = model.get_weights()
            print('Saved')
    
    fold_no = fold_no + 1
    www.append(www_temp)

print('Peaks at Epoch', peak[0], 'with accuracy', np.round(peak[1],3), 'and loss', np.round(peak[2],3))

In [ ]:
print('Peaks at Epoch', peak[0], 'with accuracy', np.round(peak[1],3), 'and loss', np.round(peak[2],3))
fig=plt.figure(figsize=(15,9))
plt.plot(kt_los,label='tr ls')
plt.plot(kt_acc,label='tr ac')
plt.plot(kv_los,label='vl ls')
plt.plot(kv_acc,label='vl ac')
plt.legend()
# plt.ylim(0.55,0.7)
plt.show()

In [ ]:
model.summary()

In [ ]:
savedpeak   

In [ ]:
def getNNpeaks(array):
    ## Takes de savedpeak list from the NN training, 
    ## gets the epoch position of the peaks, and extracts the loss and accuracy
    
    epoch_ini = array[0][0]
    epoch_1 = epoch_ini ## necessary so that the comparison works well
    peaks = []
    for i in array:
        if i[0] >= epoch_1: ## necessary so that the first element does not get arbitrarily saved
#             print(i[0],epoch_1)
            epoch_1 = i[0]  ## necessary so that the loop can work
            epoch_2_full = i   ## necessary so that the pleak can be saved in the next iteration,
                               ## when the epoch is smaller and gets saved
        else:
#             print('saved?',i[0],epoch_1)
            epoch_1 = i[0]
            peaks.append(epoch_2_full)
    peaks.append(i) ## Because the last one will never be get through the loop, but would definitely 
                    ## will be get as the last input of the array
    accu = []
    loss = []
    for i in peaks:
        accu.append(i[1])
        loss.append(i[2])
    
    return accu, loss

In [ ]:
NNaccu, NNloss = getNNpeaks(savedpeak)

In [ ]:
np.mean(NNaccu), np.mean(NNloss) # Try Jul 15 (0.69979995, 0.65599996) # Try Jul 15 again (0.69940007, 0.6548)

In [ ]:
np.mean(NNaccu), np.std(NNaccu)/np.sqrt(5)

In [ ]:
np.save('./w0_skyErr_pre-norm-log.npy',www[0])
np.save('./w1_skyErr_pre-norm-log.npy',www[1])
np.save('./w2_skyErr_pre-norm-log.npy',www[2])
np.save('./w3_skyErr_pre-norm-log.npy',www[3])
np.save('./w4_skyErr_pre-norm-log.npy',www[4])

## Steps to check the performance on the Test set

In [ ]:
www0 = np.load('./w0_skyErr_pre-norm-log.npy',allow_pickle=True)
www1 = np.load('./w1_skyErr_pre-norm-log.npy',allow_pickle=True)
www2 = np.load('./w2_skyErr_pre-norm-log.npy',allow_pickle=True)
www3 = np.load('./w3_skyErr_pre-norm-log.npy',allow_pickle=True)
www4 = np.load('./w4_skyErr_pre-norm-log.npy',allow_pickle=True)

In [ ]:
www = []
www.append(www0)
www.append(www1)
www.append(www2)
www.append(www3)
www.append(www4)

In [ ]:
test_file = '/home/phd11/Photometric_Classification//primary_countE_Inputs/ErrorInputs_fiber_test2.fits'
test_tab = prepare_data(test_file, CLASS_COLUMN, NO_CLASS, USE_COLUMNS)
test_ds = kfold_iteration(test_tab,set_type='Valid')

In [ ]:
### In order to get the accuracy on the test set, we apply the NN on it and calculate the classification types, \
### making used of the 5 save weights, corresponding to the 5 cross-validations folds.

tf.keras.backend.clear_session()
model = photo_model()
acc_test = []

### An initial run of the model is required as initialization
x_batch_ini, y_batch_ini = next(iter(test_ds))
buena_ts_ini = model(x_batch_ini, training=False)
for i in range(0,5):
    model.set_weights(www[i])
    for step in range(0, STEPS_PER_TEST_EPOCH):
        ### Although I only use 1 Test epoch, might be handy in case one is interested on modifying it
        x_batch, y_batch = next(iter(test_ds))
        buena_ts = model(x_batch, training=False)

    buena_ts = buena_ts.numpy()
    ts = buena_ts[:,0]

    TPtest = np.where((y_batch.numpy()[:,0]==0) & (buena_ts[:,0]<0.5))[0]
    FNtest = np.where((y_batch.numpy()[:,0]==0) & (buena_ts[:,0]>=0.5))[0]
    TNtest = np.where((y_batch.numpy()[:,0]==1) & (buena_ts[:,0]>=0.5))[0]

    accuracy = (len(TPtest)+ len(TNtest))/len(ts)
    acc_test.append(accuracy)
    print(accuracy, len(TPtest), len(TNtest),len(ts))

In [ ]:
print(np.mean(acc_test),np.std(acc_test)/np.sqrt(5))